### math 510 project

In [35]:
import numpy as np
import matplotlib.pyplot as plt
import random
import time
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [36]:
def median_twenty(t):
    n = len(t)
    if n <= 20:
        return t
    res = t[:20]
    for i in range(20, n):
        s = sorted(t[i - 20 : i])
        res.append(s[10])
    return res

In [37]:
# calculate dx/dt (equation 3, p. 18)
def calculate_response(x):
    return np.tanh(x)

def calculate_derivative(x, u, J, B, tau):
    r = calculate_response(x)
    return (1.0 / tau) * (-x + np.dot(J,r) + np.dot(B,u))

In [38]:
# calculate euler approximation of the time step
# def euler_timestep(x, u, J, B, tau, delta_t):
#     return x + calculate_derivative(x, u, J, B, tau) * delta_t

# hacky version to squeeze out some performance!
def euler_timestep(x, u, J, B, tau, delta_t):
    return x * 0.96666666 + (np.dot(J,np.tanh(x)) + np.dot(B,u)) * 0.0333333

In [39]:
def apply_perturbations(x, p, N):
    positions = np.random.rand(N,1) < p
    perturbations = np.random.rand(N,1) - 0.5
    perturbations[0][0] = 0.0  # don't perturb the output neuron
    return x + positions * perturbations

In [40]:
def apply_perturbations2(x, p, N):
    positions = np.random.rand(N,1) < p
    perturbations = np.random.rand(N,1) - 0.5
    # don't perturb any of the output neurons
    perturbations[0][0] = 0.0
    perturbations[1][0] = 0.0
    perturbations[2][0] = 0.0
    perturbations[3][0] = 0.0
    return x + positions * perturbations

In [41]:
# note: the choice of bias neurons is arbitrary
def enforce_biases(x):
    x[10][0] = 1.0
    x[11][0] = 1.0
    x[12][0] = 1.0
    x[13][0] = -1.0

In [42]:
def update_learning_potential(E, r_prev, x, x_average, N):
    res = np.outer(x - x_average, r_prev)
    return E + res * res * res

In [43]:
def run_trial_task_1(J, B, tau, delta_t, N, M, expected_reward, eta, plot_output = False):
    E = np.zeros((N,N))  # learning potential
    x_alpha = 0.95       # decay of short-term running average of x
    r_alpha = 0.75       # decay of expected reward
    p = 0.003            # probability of perturbation
    
    # set up the external inputs
    external = {'A': np.array([1.,0.]).reshape(2,1), 'B': np.array([0.,1.]).reshape(2,1)}
    choice1 = random.choice(['A', 'B'])
    choice2 = random.choice(['A', 'B'])
    trial_type = choice1 == choice2
    reward_trial_type = choice1 + choice2
    target_response = -1.0 if trial_type else 1.0
    if plot_output: print "Stimulus 1:", choice1
    if plot_output: print "Stimulus 2:", choice2
    u1 = external[choice1]
    u2 = external[choice2]
    u_s = [u1] * 200 + [np.zeros((M,1))] * 200 + [u2] * 200 + [np.zeros((M,1))] * 400
    
    # initialize the excitation
    x = 0.1 * (2 * np.random.rand(N,1) - 1)
    x_average = np.zeros((N,1))
    r_s = []
    
    # run the trial timesteps
    for timestep in range(1000):
        u = u_s[timestep]
        r_prev = np.tanh(x)
        r_s.append(r_prev[0][0])
        x_average = x_alpha * x + (1 - x_alpha) * x_average
        x = apply_perturbations(x, p, N)
        enforce_biases(x)
        x = euler_timestep(x, u, J, B, tau, delta_t)
        E = update_learning_potential(E, r_prev, x, x_average, N)
        
    
    # compute the error
    trial_error = sum(map(lambda x: np.abs(target_response - x), r_s[800:])) / 200.0
    prev_expected_reward = expected_reward[reward_trial_type]
    trial_reward = trial_error - prev_expected_reward
    expected_reward[reward_trial_type] = r_alpha * prev_expected_reward + (1 - r_alpha) * trial_error
    
    # return (weight_change, expected_reward, trial_error, r_s) tuple
    return (eta * trial_reward * prev_expected_reward * E, expected_reward, trial_error, r_s)

In [44]:
def task1(num_trials):
    
    N = 200         # number of neurons (200)
    M = 2           # number of external inputs to the network (2)
    T = 1           # total time of the simulation (?)
    delta_t = 0.001 # length of the time step (1 ms)
    tau = 0.03      # relaxation time constant (30 ms)
    g = 1.5         # scaling factor (1.5)
    eta = 0.1       # learning rate (0.5 paper, 0.1 code)
    
    J = np.random.normal(loc=0.0, scale=(g/np.sqrt(N)), size=(N,N))
    B = 2 * np.random.rand(N,M) - 1
    
    expected_reward = {'AA': 0.0, 'AB': 0.0, 'BA': 0.0, 'BB': 0.0}
    
    e_s = []
    t_s = []
    
    # plot the output before training
    print "BEFORE TRAINING"
    print "==============="
    _, _, _, r_s = run_trial_task_1(J, B, tau, delta_t, N, M, expected_reward, eta, plot_output = True)
    plt.plot(list(range(1000)), r_s)
    plt.title('r(x[0]) over time')
    plt.xlabel('timestep (ms)')
    plt.ylabel('response')
    plt.show()

    # print the weights
    #print
    #print J
    #print
    
    for trial in range(num_trials):
        weight_change, expected_reward, trial_error, _ = run_trial_task_1(J, B, tau, delta_t, N, M, expected_reward, eta)
        J -= np.clip(weight_change, -0.0003, 0.0003)
        e_s.append(trial_error)
        t_s.append(trial)
    
    # plot the output before training
    print "AFTER TRAINING"
    print "=============="
    _, _, _, r_s = run_trial_task_1(J, B, tau, delta_t, N, M, expected_reward, eta, plot_output = True)
    plt.plot(list(range(1000)), r_s)
    plt.title('r(x[0]) over time')
    plt.xlabel('timestep (ms)')
    plt.ylabel('response')
    plt.show()
    
    # print the weights
    #print
    #print J
    #print
    
    # plot error
    plt.plot(t_s, median_twenty(e_s))
    plt.title('median trial error over time')
    plt.xlabel('trial')
    plt.ylabel('error')
    plt.show()
    
    return

In [45]:
#%lprun -f run_trial_task_1 task1(50)
#task1(10000)

In [46]:
# alternate weight change calculation: eta * (r - r_bar) * E
def run_trial_task_1_a1(J, B, tau, delta_t, N, M, expected_reward, eta, plot_output = False):
    E = np.zeros((N,N))  # learning potential
    x_alpha = 0.95       # decay of short-term running average of x
    r_alpha = 0.75       # decay of expected reward
    p = 0.003            # probability of perturbation
    
    # set up the external inputs
    external = {'A': np.array([1.,0.]).reshape(2,1), 'B': np.array([0.,1.]).reshape(2,1)}
    choice1 = random.choice(['A', 'B'])
    choice2 = random.choice(['A', 'B'])
    trial_type = choice1 == choice2
    reward_trial_type = choice1 + choice2
    target_response = -1.0 if trial_type else 1.0
    if plot_output: print "Stimulus 1:", choice1
    if plot_output: print "Stimulus 2:", choice2
    u1 = external[choice1]
    u2 = external[choice2]
    u_s = [u1] * 200 + [np.zeros((M,1))] * 200 + [u2] * 200 + [np.zeros((M,1))] * 400
    
    # initialize the excitation
    x = 0.1 * (2 * np.random.rand(N,1) - 1)
    x_average = np.zeros((N,1))
    r_s = []
    
    # run the trial timesteps
    for timestep in range(1000):
        u = u_s[timestep]
        r_prev = np.tanh(x)
        r_s.append(r_prev[0][0])
        x_average = x_alpha * x + (1 - x_alpha) * x_average
        x = apply_perturbations(x, p, N)
        enforce_biases(x)
        x = euler_timestep(x, u, J, B, tau, delta_t)
        E = update_learning_potential(E, r_prev, x, x_average, N)
        
    
    # compute the error
    trial_error = sum(map(lambda x: np.abs(target_response - x), r_s[800:])) / 200.0
    prev_expected_reward = expected_reward[reward_trial_type]
    trial_reward = trial_error - prev_expected_reward
    expected_reward[reward_trial_type] = r_alpha * prev_expected_reward + (1 - r_alpha) * trial_error
    
    # return (weight_change, expected_reward, trial_error, r_s) tuple
    return (eta * trial_reward * E, expected_reward, trial_error, r_s)

In [47]:
def task1_a1(num_trials):
    
    N = 200         # number of neurons (200)
    M = 2           # number of external inputs to the network (2)
    T = 1           # total time of the simulation (?)
    delta_t = 0.001 # length of the time step (1 ms)
    tau = 0.03      # relaxation time constant (30 ms)
    g = 1.5         # scaling factor (1.5)
    eta = 0.1       # learning rate (0.5 paper, 0.1 code)
    
    J = np.random.normal(loc=0.0, scale=(g/np.sqrt(N)), size=(N,N))
    B = 2 * np.random.rand(N,M) - 1
    
    expected_reward = {'AA': 0.0, 'AB': 0.0, 'BA': 0.0, 'BB': 0.0}
    
    e_s = []
    t_s = []
    
    # plot the output before training
    print "BEFORE TRAINING"
    print "==============="
    _, _, _, r_s = run_trial_task_1_a1(J, B, tau, delta_t, N, M, expected_reward, eta, plot_output = True)
    plt.plot(list(range(1000)), r_s)
    plt.title('r(x[0]) over time')
    plt.xlabel('timestep (ms)')
    plt.ylabel('response')
    plt.show()

    # print the weights
    #print
    #print J
    #print
    
    for trial in range(num_trials):
        weight_change, expected_reward, trial_error, _ = run_trial_task_1_a1(J, B, tau, delta_t, N, M, expected_reward, eta)
        J -= np.clip(weight_change, -0.0003, 0.0003)
        e_s.append(trial_error)
        t_s.append(trial)
    
    # plot the output before training
    print "AFTER TRAINING"
    print "=============="
    _, _, _, r_s = run_trial_task_1_a1(J, B, tau, delta_t, N, M, expected_reward, eta, plot_output = True)
    plt.plot(list(range(1000)), r_s)
    plt.title('r(x[0]) over time')
    plt.xlabel('timestep (ms)')
    plt.ylabel('response')
    plt.show()
    
    # print the weights
    #print
    #print J
    #print
    
    # plot error
    plt.plot(t_s, median_twenty(e_s))
    plt.title('median trial error over time')
    plt.xlabel('trial')
    plt.ylabel('error')
    plt.show()
    
    return

In [48]:
#task1_a1(10000)

In [49]:
# two cumulative reward types: match (AA, BB) or no match (AB, BA)
def run_trial_task_1_a2(J, B, tau, delta_t, N, M, expected_reward, eta, plot_output = False):
    E = np.zeros((N,N))  # learning potential
    x_alpha = 0.95       # decay of short-term running average of x
    r_alpha = 0.75       # decay of expected reward
    p = 0.003            # probability of perturbation
    
    # set up the external inputs
    external = {'A': np.array([1.,0.]).reshape(2,1), 'B': np.array([0.,1.]).reshape(2,1)}
    choice1 = random.choice(['A', 'B'])
    choice2 = random.choice(['A', 'B'])
    trial_type = choice1 == choice2
    reward_trial_type = trial_type
    target_response = -1.0 if trial_type else 1.0
    if plot_output: print "Stimulus 1:", choice1
    if plot_output: print "Stimulus 2:", choice2
    u1 = external[choice1]
    u2 = external[choice2]
    u_s = [u1] * 200 + [np.zeros((M,1))] * 200 + [u2] * 200 + [np.zeros((M,1))] * 400
    
    # initialize the excitation
    x = 0.1 * (2 * np.random.rand(N,1) - 1)
    x_average = np.zeros((N,1))
    r_s = []
    
    # run the trial timesteps
    for timestep in range(1000):
        u = u_s[timestep]
        r_prev = np.tanh(x)
        r_s.append(r_prev[0][0])
        x_average = x_alpha * x + (1 - x_alpha) * x_average
        x = apply_perturbations(x, p, N)
        enforce_biases(x)
        x = euler_timestep(x, u, J, B, tau, delta_t)
        E = update_learning_potential(E, r_prev, x, x_average, N)
        
    
    # compute the error
    trial_error = sum(map(lambda x: np.abs(target_response - x), r_s[800:])) / 200.0
    prev_expected_reward = expected_reward[reward_trial_type]
    trial_reward = trial_error - prev_expected_reward
    expected_reward[reward_trial_type] = r_alpha * prev_expected_reward + (1 - r_alpha) * trial_error
    
    # return (weight_change, expected_reward, trial_error, r_s) tuple
    return (eta * trial_reward * prev_expected_reward * E, expected_reward, trial_error, r_s)

In [50]:
def task1_a2(num_trials):
    
    N = 200         # number of neurons (200)
    M = 2           # number of external inputs to the network (2)
    T = 1           # total time of the simulation (?)
    delta_t = 0.001 # length of the time step (1 ms)
    tau = 0.03      # relaxation time constant (30 ms)
    g = 1.5         # scaling factor (1.5)
    eta = 0.1       # learning rate (0.5 paper, 0.1 code)
    
    J = np.random.normal(loc=0.0, scale=(g/np.sqrt(N)), size=(N,N))
    B = 2 * np.random.rand(N,M) - 1
    
    expected_reward = {True: 0.0, False: 0.0}
    
    e_s = []
    t_s = []
    
    # plot the output before training
    print "BEFORE TRAINING"
    print "==============="
    _, _, _, r_s = run_trial_task_1_a2(J, B, tau, delta_t, N, M, expected_reward, eta, plot_output = True)
    plt.plot(list(range(1000)), r_s)
    plt.title('r(x[0]) over time')
    plt.xlabel('timestep (ms)')
    plt.ylabel('response')
    plt.show()

    # print the weights
    #print
    #print J
    #print
    
    for trial in range(num_trials):
        weight_change, expected_reward, trial_error, _ = run_trial_task_1_a2(J, B, tau, delta_t, N, M, expected_reward, eta)
        J -= np.clip(weight_change, -0.0003, 0.0003)
        e_s.append(trial_error)
        t_s.append(trial)
    
    # plot the output before training
    print "AFTER TRAINING"
    print "=============="
    _, _, _, r_s = run_trial_task_1_a2(J, B, tau, delta_t, N, M, expected_reward, eta, plot_output = True)
    plt.plot(list(range(1000)), r_s)
    plt.title('r(x[0]) over time')
    plt.xlabel('timestep (ms)')
    plt.ylabel('response')
    plt.show()
    
    # print the weights
    #print
    #print J
    #print
    
    # plot error
    plt.plot(t_s, median_twenty(e_s))
    plt.title('median trial error over time')
    plt.xlabel('trial')
    plt.ylabel('error')
    plt.show()
    
    return

In [51]:
#task1_a2(10000)

In [52]:
# discrete reward
def discretize_reward(r):
    return 0.0 if r < 0.05 else 1.0
    
def run_trial_task_1_d(J, B, tau, delta_t, N, M, expected_reward, eta, plot_output = False):
    E = np.zeros((N,N))  # learning potential
    x_alpha = 0.95       # decay of short-term running average of x
    r_alpha = 0.75       # decay of expected reward
    p = 0.003            # probability of perturbation
    
    # set up the external inputs
    external = {'A': np.array([1.,0.]).reshape(2,1), 'B': np.array([0.,1.]).reshape(2,1)}
    choice1 = random.choice(['A', 'B'])
    choice2 = random.choice(['A', 'B'])
    trial_type = choice1 == choice2
    reward_trial_type = choice1 + choice2
    target_response = -1.0 if trial_type else 1.0
    if plot_output: print "Stimulus 1:", choice1
    if plot_output: print "Stimulus 2:", choice2
    u1 = external[choice1]
    u2 = external[choice2]
    u_s = [u1] * 200 + [np.zeros((M,1))] * 200 + [u2] * 200 + [np.zeros((M,1))] * 400
    
    # initialize the excitation
    x = 0.1 * (2 * np.random.rand(N,1) - 1)
    x_average = np.zeros((N,1))
    r_s = []
    
    # run the trial timesteps
    for timestep in range(1000):
        u = u_s[timestep]
        r_prev = np.tanh(x)
        r_s.append(r_prev[0][0])
        x_average = x_alpha * x + (1 - x_alpha) * x_average
        x = apply_perturbations(x, p, N)
        enforce_biases(x)
        x = euler_timestep(x, u, J, B, tau, delta_t)
        E = update_learning_potential(E, r_prev, x, x_average, N)
        
    
    # compute the error
    trial_error = sum(map(lambda x: np.abs(target_response - x), r_s[800:])) / 200.0
    discrete_error = discretize_reward(trial_error)
    prev_expected_reward = expected_reward[reward_trial_type]
    trial_reward = discrete_error - prev_expected_reward
    expected_reward[reward_trial_type] = r_alpha * prev_expected_reward + (1 - r_alpha) * trial_error
    
    # return (weight_change, expected_reward, trial_error, r_s) tuple
    return (eta * trial_reward * prev_expected_reward * E, expected_reward, trial_error, r_s)

In [53]:
def task1_d(num_trials):
    
    N = 200         # number of neurons (200)
    M = 2           # number of external inputs to the network (2)
    T = 1           # total time of the simulation (?)
    delta_t = 0.001 # length of the time step (1 ms)
    tau = 0.03      # relaxation time constant (30 ms)
    g = 1.5         # scaling factor (1.5)
    eta = 0.1       # learning rate (0.5 paper, 0.1 code)
    
    J = np.random.normal(loc=0.0, scale=(g/np.sqrt(N)), size=(N,N))
    B = 2 * np.random.rand(N,M) - 1
    
    expected_reward = {'AA': 0.0, 'AB': 0.0, 'BA': 0.0, 'BB': 0.0}
    
    e_s = []
    t_s = []
    
    # plot the output before training
    print "BEFORE TRAINING"
    print "==============="
    _, _, _, r_s = run_trial_task_1_d(J, B, tau, delta_t, N, M, expected_reward, eta, plot_output = True)
    plt.plot(list(range(1000)), r_s)
    plt.title('r(x[0]) over time')
    plt.xlabel('timestep (ms)')
    plt.ylabel('response')
    plt.show()

    # print the weights
    #print
    #print J
    #print
    
    for trial in range(num_trials):
        weight_change, expected_reward, trial_error, _ = run_trial_task_1_d(J, B, tau, delta_t, N, M, expected_reward, eta)
        J -= np.clip(weight_change, -0.0003, 0.0003)
        e_s.append(trial_error)
        t_s.append(trial)
    
    # plot the output before training
    print "AFTER TRAINING"
    print "=============="
    _, _, _, r_s = run_trial_task_1_d(J, B, tau, delta_t, N, M, expected_reward, eta, plot_output = True)
    plt.plot(list(range(1000)), r_s)
    plt.title('r(x[0]) over time')
    plt.xlabel('timestep (ms)')
    plt.ylabel('response')
    plt.show()
    
    # print the weights
    #print
    #print J
    #print
    
    # plot error
    plt.plot(t_s, median_twenty(e_s))
    plt.title('median trial error over time')
    plt.xlabel('trial')
    plt.ylabel('error')
    plt.show()
    
    return

In [54]:
#task1_d(10000)

In [55]:
def get_predicted_trial_type(t):
    max_comps = [l.index(max(l)) for l in t]
    return max(set(max_comps), key=max_comps.count)


def run_trial_task_2(J, B, tau, delta_t, N, M, expected_reward, eta, plot_output = False):
    E = np.zeros((N,N))  # learning potential
    x_alpha = 0.95       # decay of short-term running average of x
    r_alpha = 0.75       # decay of expected reward
    p = 0.003            # probability of perturbation
    
    # set up the external inputs
    external = {'A': np.array([1.,0.]).reshape(2,1), 'B': np.array([0.,1.]).reshape(2,1)}
    target_response_lookup = {'AA': (1.0,0.0,0.0,0.0), 'AB': (0.0,1.0,0.0,0.0), 'BA': (0.0,0.0,1.0,0.0), 'BB': (0.0,0.0,0.0,1.0)}
    trial_type_lookup = {'AA': 0, 'AB': 1, 'BA': 2, 'BB': 3}
    choice1 = random.choice(['A', 'B'])
    choice2 = random.choice(['A', 'B'])
    #trial_type = choice1 == choice2
    reward_trial_type = choice1 + choice2
    target_response = target_response_lookup[reward_trial_type]
    if plot_output: print "Stimulus 1:", choice1
    if plot_output: print "Stimulus 2:", choice2
    u1 = external[choice1]
    u2 = external[choice2]
    u_s = [u1] * 200 + [np.zeros((M,1))] * 200 + [u2] * 200 + [np.zeros((M,1))] * 400
    
    # initialize the excitation
    x = 0.1 * (2 * np.random.rand(N,1) - 1)
    x_average = np.zeros((N,1))
    r_s = []
    
    # run the trial timesteps
    for timestep in range(1000):
        u = u_s[timestep]
        r_prev = np.tanh(x)
        r_s.append((r_prev[0][0],r_prev[1][0],r_prev[2][0],r_prev[3][0]))
        x_average = x_alpha * x + (1 - x_alpha) * x_average
        x = apply_perturbations2(x, p, N)
        enforce_biases(x)
        x = euler_timestep(x, u, J, B, tau, delta_t)
        E = update_learning_potential(E, r_prev, x, x_average, N)
        
    
    # compute the error
    predicted_tt = get_predicted_trial_type(r_s[800:])
    rs_by_component = map(lambda t: t[800:], zip(*r_s))
    abs_diff_from_zero = map(lambda t: sum(map(lambda x: np.abs(x), t)) / 200.0, rs_by_component)
    trial_error = sum(map(lambda x: np.abs(1 - x), rs_by_component[predicted_tt])) / 200.0
    prev_expected_reward = expected_reward[trial_type_lookup[reward_trial_type]]
    trial_reward = trial_error - prev_expected_reward
    for i in range(4):
        expected_reward[i] = abs_diff_from_zero[i] * (r_alpha * prev_expected_reward + (1 - r_alpha) * trial_error)
    # return (weight_change, expected_reward, trial_error, r_s) tuple
    return (eta * trial_reward * prev_expected_reward * E, expected_reward, trial_error, r_s)

In [56]:
def task2(num_trials):
    
    N = 200         # number of neurons (200)
    M = 2           # number of external inputs to the network (2)
    T = 1           # total time of the simulation (?)
    delta_t = 0.001 # length of the time step (1 ms)
    tau = 0.03      # relaxation time constant (30 ms)
    g = 1.5         # scaling factor (1.5)
    eta = 0.1       # learning rate (0.5 paper, 0.1 code)
    
    J = np.random.normal(loc=0.0, scale=(g/np.sqrt(N)), size=(N,N))
    B = 2 * np.random.rand(N,M) - 1
    
    expected_reward = {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}
    
    e_s = []
    t_s = []
    
    # plot the output before training
    print "BEFORE TRAINING"
    print "==============="
    _, _, _, r_s = run_trial_task_2(J, B, tau, delta_t, N, M, expected_reward, eta, plot_output = True)
    r_s = zip(*r_s)
    plt.plot(list(range(1000)), r_s[0], label='AA')
    plt.plot(list(range(1000)), r_s[1], label='AB')
    plt.plot(list(range(1000)), r_s[2], label='BA')
    plt.plot(list(range(1000)), r_s[3], label='BB')
    plt.title('output neurons over time')
    plt.xlabel('timestep (ms)')
    plt.ylabel('response')
    plt.legend(framealpha=0.5)
    plt.show()

    # print the weights
    #print
    #print J
    #print
    
    for trial in range(num_trials):
        weight_change, expected_reward, trial_error, _ = run_trial_task_2(J, B, tau, delta_t, N, M, expected_reward, eta)
        J -= np.clip(weight_change, -0.0003, 0.0003)
        e_s.append(trial_error)
        t_s.append(trial)
    
    # plot the output before training
    print "AFTER TRAINING"
    print "=============="
    _, _, _, r_s = run_trial_task_2(J, B, tau, delta_t, N, M, expected_reward, eta, plot_output = True)
    r_s = zip(*r_s)
    plt.plot(list(range(1000)), r_s[0], label='AA')
    plt.plot(list(range(1000)), r_s[1], label='AB')
    plt.plot(list(range(1000)), r_s[2], label='BA')
    plt.plot(list(range(1000)), r_s[3], label='BB')
    plt.title('output neurons over time')
    plt.xlabel('timestep (ms)')
    plt.ylabel('response')
    plt.legend(framealpha=0.5)
    plt.show()
    
    # print the weights
    #print
    #print J
    #print
    
    # plot error
    plt.plot(t_s, median_twenty(e_s))
    plt.title('median trial error over time')
    plt.xlabel('trial')
    plt.ylabel('error')
    plt.show()
    
    return

In [57]:
#task2(10000)